# Getting Started

Let's find out the basic usage cases for the **netallocation** package! 

You can install the package via [pypi](https://pypi.org/project/netallocation/) with
    
```
pip install netallocation
```

and run this notebook locally on your computer. 

In [ ]:
import netallocation as ntl
import matplotlib.pyplot as plt

We want to have a look at a small *toy* pypsa network

In [ ]:
n = ntl.test.get_network_ac_dc()

This network has the following N = 9 buses:

In [ ]:
n.buses

and L = 11 branches

In [ ]:
ntl.utils.check_carriers(n)
n.branches()[['bus0', 'bus1', 'carrier']]

The network consists of 3 passive DC and 7 passive AC lines, there is one controllable HVDC link from London to Bremen. The **capacities**, **dispatch** and **flow** of the network is already optimized, let's see how the network looks like: 

In [ ]:
plot_kwargs = dict(line_colors=n.branches().carrier.map({'DC': 'green', 'AC': 'purple'}), color_geomap=True)

fig = plt.figure(figsize=(9,9))
n.plot(bus_sizes=n.generators.groupby(['bus', 'carrier']).p_nom_opt.sum()/5e3, margin=.5, 
       title='Wind - blue, Gas - red, AC - purple, DC - green', **plot_kwargs);

# Allocating Flows

Let's have a closer look at the **first** snapshot

In [ ]:
sn = n.snapshots[0]
fig = plt.figure(figsize=(9,9))
n.plot(flow=sn, line_widths={'Line':.1, 'Link':.1}, **plot_kwargs);

We are interested in the flow originating from the bus in **Norway**. Which way does is flow through the network? Where does its power flow to? 

Therefore we **allocate** the flow using the *flow tracing* (Average participation) method using the function `allocate_flow()`

In [ ]:
ap = ntl.allocate_flow(n, snapshots=sn, method='Average participation')
ap

There are two variables, `peer_to_peer` of dimension **source** $\times$ **sink** and `peer_on_branch_to_peer` of dimension **branch** $\times$ **source** $\times$ **sink**. With the first one, we can have look where **Norway** delivers power.

In [ ]:
ap.peer_to_peer.sel(source='Norway').to_series()[lambda ds: abs(ds)>=1e-7]

With the second we can trace the flow originating from **Norway**:

In [ ]:
flow = ap.peer_on_branch_to_peer.sel(source='Norway').sum('sink')
n.plot(flow=flow.to_series()/10, **plot_kwargs);

Is this concistent with the `P2P` allocation? This can be checked by summing over all power flowing into a bus (Incidence matrix!) 

In [ ]:
(ntl.Incidence(n) @ flow).to_series()[lambda ds: abs(ds)>=1e-7]

# Allocating $CO_2$-emissions

In [ ]:
co2_allocation = ntl.cost.weight_with_carrier_attribute(n, 'ap', attr='co2_emissions')

In [ ]:
co2_flow = co2_allocation.peer_on_branch_to_peer.sum(['snapshot', 'source', 'sink', 'source_carrier'])
n.plot(flow = co2_flow.to_series(), jitter=.7, margin=.2, title='Average Carbon Flow', **plot_kwargs);